In [1]:
class KnapsackOptimal:
    def __init__(self, bag_max_weight, item_weights, item_values):
        self.solution_values = {}
        self.bag_max_weight = bag_max_weight
        self.item_weights = item_weights
        self.item_values = item_values
        self.number_of_items = len(item_weights)
    
    def solve(self):
        return self._solver(self.bag_max_weight,
                            self.item_weights,
                            self.item_values,
                            self.number_of_items)
    
    def _solver(self, W, w, v, n):
        # print('_solver', W, w, v, n)
        if n == 0 or W == 0:
            return 0

        if w[n-1] > W:
            return self._solver(W, w, v, n-1)
        
        if (W, n) in self.solution_values:
            return self.solution_values[(W, n)]
        
        value_with_item = v[n-1] + self._solver(W - w[n-1], w, v, n-1)
        value_without_item = self._solver(W, w, v, n-1)
        best_value = max(value_with_item, value_without_item)
        
        self.solution_values[(W, n)] = best_value
        
        return best_value
    
    
def solve_optimal_knapsack(bag_max_weight, item_weights, item_values):
    algo = KnapsackOptimal(bag_max_weight, item_weights, item_values)
    answer = algo.solve()
    print(f'Solve W={bag_max_weight} w={item_weights} v={item_values} n={len(item_weights)}:', answer)


solve_optimal_knapsack(20, [6, 14], [30, 1])
solve_optimal_knapsack(20, [6, 15], [30, 1])
solve_optimal_knapsack(20, [1, 2, 3, 4, 5, 6], [6, 5, 4, 3, 2, 1])

Solve W=20 w=[6, 14] v=[30, 1] n=2: 31
Solve W=20 w=[6, 15] v=[30, 1] n=2: 30
Solve W=20 w=[1, 2, 3, 4, 5, 6] v=[6, 5, 4, 3, 2, 1] n=6: 20


In [2]:
max_weight = 30
item_weights = [7, 2, 1, 9, 5, 4, 3, 6, 8]
item_values =  [5, 4, 7, 2, 6, 1, 3, 8, 9]

In [3]:
import random

def generate_population(count):
    result = set()
    while len(result) < count:
        new_individual = tuple(random.choices([0, 1], k=len(item_weights)))
        result.add(new_individual)
    
    return list(result)

POPULATION_SIZE = 10
population = generate_population(POPULATION_SIZE)
population

[(0, 1, 1, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 1, 1),
 (1, 0, 1, 0, 1, 0, 0, 0, 1),
 (0, 1, 1, 0, 1, 0, 0, 1, 1),
 (0, 1, 1, 1, 0, 0, 1, 1, 1),
 (0, 1, 1, 1, 1, 1, 0, 0, 1),
 (1, 1, 0, 1, 1, 1, 0, 0, 1),
 (0, 0, 1, 0, 0, 1, 0, 0, 1),
 (1, 0, 0, 1, 1, 1, 1, 0, 1),
 (0, 1, 0, 0, 1, 1, 0, 1, 1)]

In [4]:
def calc_fitness(indiv):
    total_value, total_weight = 0, 0
    for idx, flag in enumerate(indiv):
        total_value += item_values[idx] if flag else 0
        total_weight += item_weights[idx] if flag else 0
    
    return total_value if total_weight <= max_weight else 0

# calc_fitness((1, 1, 1, 0))

In [5]:
import copy

def selection(population):
    parents = []
    
    population = copy.deepcopy(population)
    random.shuffle(population)
    
    parents.append(population[0]
                   if calc_fitness(population[0]) > calc_fitness(population[1])
                   else population[1])
    parents.append(population[2]
                   if calc_fitness(population[2]) > calc_fitness(population[3])
                   else population[3])
    
    return parents

parents = selection(population)
print(parents)

[(1, 0, 1, 0, 1, 0, 0, 0, 1), (0, 1, 1, 1, 0, 0, 1, 1, 1)]


In [6]:
def crossover(parents):
    assert len(parents) == 2
    # print(parents[0], parents[1])
    # print(5 // 2)
    split = len(item_values) // 2
    child1 = parents[0][:split] + parents[1][split:]
    child2 = parents[0][split:] + parents[1][:split]
    
    return [child1, child2]

crossover(parents)

[(1, 0, 1, 0, 0, 0, 1, 1, 1), (1, 0, 0, 0, 1, 0, 1, 1, 1)]

In [7]:
MUTATION_RATE = 0.07
MUTATION_INTENSITY = 0.15
def mutate(offsprings):
    
    def _mutate_single(ind):
        new_bits = []
        inv_map = {0: 1, 1: 0}
        for bit in ind:
            new_bits.append(inv_map[bit]
                            if random.random() < MUTATION_INTENSITY
                            else bit)
        # print(f'mutate {ind} -> {tuple(new_bits)}')
        return tuple(new_bits)
    
    result = []
    result.append(_mutate_single(offsprings[0])
                  if random.random() < MUTATION_RATE
                  else offsprings[0])
    result.append(_mutate_single(offsprings[1])
                  if random.random() < MUTATION_RATE
                  else offsprings[1])
    
    return result


In [8]:
for ind in population:
    print(f'indiv: {ind} / best_fitness: {calc_fitness(ind)}')

indiv: (0, 1, 1, 0, 1, 0, 0, 0, 0) / best_fitness: 17
indiv: (0, 0, 0, 1, 0, 0, 0, 1, 1) / best_fitness: 19
indiv: (1, 0, 1, 0, 1, 0, 0, 0, 1) / best_fitness: 27
indiv: (0, 1, 1, 0, 1, 0, 0, 1, 1) / best_fitness: 34
indiv: (0, 1, 1, 1, 0, 0, 1, 1, 1) / best_fitness: 33
indiv: (0, 1, 1, 1, 1, 1, 0, 0, 1) / best_fitness: 29
indiv: (1, 1, 0, 1, 1, 1, 0, 0, 1) / best_fitness: 0
indiv: (0, 0, 1, 0, 0, 1, 0, 0, 1) / best_fitness: 17
indiv: (1, 0, 0, 1, 1, 1, 1, 0, 1) / best_fitness: 0
indiv: (0, 1, 0, 0, 1, 1, 0, 1, 1) / best_fitness: 28


In [9]:
import copy

best_fitness = -1000
for i in range(100):
    offsprings = []
    while len(offsprings) < POPULATION_SIZE:
        parents = selection(population)
        new_offsprings = crossover(parents)
        # new_offsprings = mutate(new_offsprings)
        offsprings.extend(new_offsprings)
    
    new_population = []
    new_population.extend(population)
    new_population.extend(offsprings)
    new_population.sort(key=lambda i: calc_fitness(i), reverse=True)
    # for ind in new_population:
    #     print(ind, calc_fitness(ind))
    new_population = new_population[:POPULATION_SIZE]
    population = new_population
    best_fitness = max(best_fitness, calc_fitness(population[0]))
    # print(f'idx: {i} / indiv: {population[0]} / best_fitness: {best_fitness}')


print('best_fitness:', best_fitness)

for ind in population:
    print(f'indiv: {ind} / best_fitness: {calc_fitness(ind)}')

best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
indiv: (1, 0, 1, 0, 1, 0, 1, 1, 1) / best_fitness: 38
